### How Are We Doing?

In the last notebook, you created a working version of SVD for situations even when there are tons of missing values.  This is awesome!  The question now is how well does this solution work?

In this notebook, we are going to simulate exactly what we would do in the real world to tune our recommender.  

Run the cell below to read in the data and get started.

In [2]:
import numpy as np
import pandas as pd

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

1. Using the **reviews** dataframe, perform the following tasks to create a training and validation set of data we can use to test the performance of your SVD algorithm using **off-line** validation techniques.

 * Order the reviews dataframe from earliest to most recent 
 * Pull the first 10000 reviews from  the dataset
 * Make the first 8000/10000 reviews the training data 
 * Make the last 2000/10000 the test data
 * Return the training and test datasets

In [2]:
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [3]:
def create_train_test(reviews, order_by, training_size, testing_size):
    '''    
    INPUT:
    reviews - (pandas df) dataframe to split into train and test
    order_by - (string) column name to sort by
    training_size - (int) number of rows in training set
    testing_size - (int) number of columns in the test set
    
    OUTPUT:
    training_df -  (pandas df) dataframe of the training set
    validation_df - (pandas df) dataframe of the test set
    '''
    reviews = reviews.sort_values(order_by,ascending=True)
    reviews_subset = reviews[: (training_size+testing_size)]
    training_df = reviews_subset[:training_size]
    validation_df = reviews_subset[training_size: (training_size+testing_size)]
    
    return training_df, validation_df

In [4]:
# Nothing to change in this or the next cell
# Use our function to create training and test datasets
train_df, val_df = create_train_test(reviews, 'date', 8000, 2000)

In [6]:
# Make sure the dataframes we are using are the right shape
assert train_df.shape[0] == 8000, "The number of rows doesn't look right in the training dataset."
assert val_df.shape[0] == 2000, "The number of rows doesn't look right in the validation dataset"
assert str(train_df.tail(1)['date']).split()[1] == '2013-03-15', "The last date in the training dataset doesn't look like what we expected."
assert str(val_df.tail(1)['date']).split()[1] == '2013-03-18', "The last date in the validation dataset doesn't look like what we expected."
print("Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.")

Nice job!  Looks like you have written a function that provides training and validation dataframes for you to use in the next steps.


In the real world, we might have all of the data up to this final date in the training data.  Then we want to see how well we are doing for each of the new ratings, which show up in the test data.

Below is a working example of the function created in the previous example you can use (or you can replace with your own).

`2.`  Fit the function to the training data with the following hyperparameters: 15 latent features, a learning rate of 0.005, and 250 iterations. This will take some time to run, so you may choose fewer latent features, a higher learning rate, or fewer iteratios if you want to speed up the process.  

**Note:** Again, this might be a good time to take a phone call, go for a walk, or just take a little break.  No need to change the code below unless you would like to make changes to reduce the time needed to obtain results.

In [6]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]# number of rows in the matrix
    n_movies = ratings_mat.shape[1]# number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))# total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values|
    user_mat = np.random.rand(n_users, latent_features)# user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies)# movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iter_cnt in range(iters):
        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                # if the rating exists
                    if str(ratings_mat[i,j]) != 'nan':
                        # compute the error as the actual minus the dot product of the user and movie latent features
                        error = ratings_mat[i,j] - np.dot(user_mat[i],movie_mat[:,j])
                        # Keep track of the total sum of squared errors for the matrix
                        sse_accum += error**2
                        user_new = user_mat[i] + 2 * learning_rate * error * movie_mat[:,j]
                        movie_new = movie_mat[:,j] + 2 * learning_rate * error * user_mat[i]
                        # update the values in each matrix - in the direction of the gradient
                        user_mat[i], movie_mat[:,j] = user_new, movie_new
        #print results
        break
        print(iter_cnt, sse_accum/num_ratings)
    return user_mat, movie_mat 

In [7]:
# Create user-by-item matrix - nothing to do here
train_user_item = train_df[['user_id', 'movie_id', 'rating', 'timestamp']]
train_data_df = train_user_item.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
train_data_np = np.array(train_data_df)

# Fit FunkSVD with the specified hyper parameters to the training data
user_mat, movie_mat = FunkSVD(train_data_np, latent_features=15, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 


In [12]:
train_data_df

movie_id,2844,8133,13427,14142,14538,14872,15163,15324,15400,17925,...,2625030,2635622,2638984,2645104,2645164,2646378,2649128,2735466,2748368,2750600
user_id,,,,,,,,,,,,,,,,,,,,,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
train_data_df.columns

Int64Index([   2844,    8133,   13427,   14142,   14538,   14872,   15163,
              15324,   15400,   17925,
            ...
            2625030, 2635622, 2638984, 2645104, 2645164, 2646378, 2649128,
            2735466, 2748368, 2750600],
           dtype='int64', name='movie_id', length=2679)

In [11]:
train_data_df.index

Int64Index([    8,    46,    48,    51,    66,    72,    90,   104,   112,
              130,
            ...
            53744, 53793, 53795, 53805, 53838, 53856, 53932, 53943, 53962,
            53966],
           dtype='int64', name='user_id', length=3278)

In [19]:
user_mat

array([[0.44128575, 0.77634812, 0.88761289, ..., 0.79859932, 0.59820155,
        0.20988484],
       [0.79499213, 0.55934782, 1.19044979, ..., 0.29789725, 0.83674288,
        1.15090987],
       [0.60317203, 0.41384285, 0.51656842, ..., 0.81218153, 0.65602228,
        0.44391374],
       ...,
       [0.20569188, 0.54780478, 0.78770813, ..., 0.4462466 , 0.88319107,
        0.25310999],
       [1.13173183, 0.4343564 , 1.17161801, ..., 0.74132962, 0.7872236 ,
        0.65401606],
       [0.2099716 , 0.72730277, 0.68592508, ..., 0.45751599, 0.47803644,
        1.18048287]])

Now that you have created the **user_mat** and **movie_mat**, we can use this to make predictions for how users would rate movies, by just computing the dot product of the row associated with a user and the column associated with the movie.

`3.` Use the comments in the function below to complete the **predict_rating** function.

In [13]:
def predict_rating(user_matrix, movie_matrix, user_id, movie_id):
    '''
    INPUT:
    user_matrix - user by latent factor matrix
    movie_matrix - latent factor by movie matrix
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    
    OUTPUT:
    pred - the predicted rating for user_id-movie_id according to FunkSVD
    '''
    # Create series of users and movies in the right order
    user_ids_series = np.array(train_data_df.index)
    movie_ids_series = np.array(train_data_df.columns)
    
    # User row and Movie Column
    user_row = np.where(user_ids_series == user_id)[0][0]
    movie_col = np.where(movie_ids_series == movie_id)[0][0]
    
    # Take dot product of that row and column in U and V to make prediction
    pred = np.dot(user_matrix[user_row, :], movie_matrix[:, movie_col])
    
    return pred

In [22]:
movies.index[movies.movie_id==2844][0]

15

In [18]:
user_ids_series = np.array(train_data_df.index)

In [19]:
user_ids_series

array([    8,    46,    48, ..., 53943, 53962, 53966], dtype=int64)

In [23]:
np.where(user_ids_series == 8)

(array([0], dtype=int64),)

In [24]:
movie_mat[:,[movies.index[movies.movie_id==2844][0]]]

array([[0.67086633],
       [0.3452092 ],
       [0.63214215],
       [0.90646089],
       [0.66908923],
       [0.90975648],
       [0.64466497],
       [0.32810754],
       [0.93339365],
       [0.13617354],
       [1.01396139],
       [0.34420274],
       [0.92091071],
       [0.15222831],
       [0.64636074]])

In [14]:
# Test your function with the first user-movie in the user-movie matrix (notice this is a nan)
pred_val = predict_rating(user_mat, movie_mat, 8, 2844)
pred_val

3.403998444117533

It is great that you now have a way to make predictions. However it might be nice to get a little phrase back about the user, movie, and rating.

`4.` Use the comments in the function below to complete the **predict_rating** function.  

**Note:** The movie name doesn't come back in a great format, so you can see in the solution I messed around with it a bit just to make it a little nicer.

In [15]:
def print_prediction_summary(user_id, movie_id, prediction):
    '''
    INPUT:
    user_id - the user_id from the reviews df
    movie_id - the movie_id according the movies df
    prediction - the predicted rating for user_id-movie_id
    
    OUTPUT:
    None - prints a statement about the user, movie, and prediction made
    
    '''
    movie_name = movies.movie[movies.movie_id==movie_id]
    movie_name = movie_name.replace('\nName: movie, dtype: object', '')
    print("For user {} we predict a {} rating for the movie {}.".format(user_id, round(prediction, 2), str(movie_name)))

In [16]:
# Test your function the the results of the previous function
print_prediction_summary(8, 2844, pred_val)

For user 8 we predict a 3.4 rating for the movie 15    Fantômas - À l'ombre de la guillotine (1913)
Name: movie, dtype: object.


Now that we have the ability to make predictions, let's see how well our predictions do on the test ratings we already have.  This will give an indication of how well have captured the latent features, and our ability to use the latent features to make predictions in the future!

`5.` For each of the user-movie rating in the **val_df** dataset, compare the actual rating given to the prediction you would make.  How do your predictions do?  Do you run into any problems?  If yes, what is the problem?  Use the document strings and comments below to assist as you work through these questions.

In [24]:
def validation_comparison(val_df, num_preds):
    '''
    INPUT:
    val_df - the validation dataset created in the third cell above
    num_preds - (int) the number of rows (going in order) you would like to make predictions for
    
    OUTPUT:
    Nothing returned - print a statement about the prediciton made for each row of val_df from row 0 to num_preds
    '''
    
    val_users = np.array(val_df['user_id'])
    val_movies = np.array(val_df['movie_id'])
    val_ratings = np.array(val_df['rating'])
    
    for i in range(num_preds):
        print(val_users[i], val_movies[i])
        pred = predict_rating(user_mat, movie_mat, val_users[i], val_movies[i])
        print("The actual rating for user {} on movie {} is {}.\n While the predicted rating is {}.".format(val_users[i], val_movies[i], val_ratings[i], round(pred))) 
    
        
# Perform the predicted vs. actual for the first 6 rows.  How does it look?
validation_comparison(val_df, 6)        

49056 1598822
The actual rating for user 49056 on movie 1598822 is 8.
 While the predicted rating is 5.
49056 289879
The actual rating for user 49056 on movie 289879 is 9.
 While the predicted rating is 5.
49056 1563738
The actual rating for user 49056 on movie 1563738 is 9.
 While the predicted rating is 6.
49056 1458175
The actual rating for user 49056 on movie 1458175 is 4.
 While the predicted rating is 5.
28599 103639
The actual rating for user 28599 on movie 103639 is 8.
 While the predicted rating is 4.
50593 1560985
The actual rating for user 50593 on movie 1560985 is 4.
 While the predicted rating is 4.


In [25]:
# Perform the predicted vs. actual for the first 7 rows.  What happened?
validation_comparison(val_df, 7)        

49056 1598822
The actual rating for user 49056 on movie 1598822 is 8.
 While the predicted rating is 5.
49056 289879
The actual rating for user 49056 on movie 289879 is 9.
 While the predicted rating is 5.
49056 1563738
The actual rating for user 49056 on movie 1563738 is 9.
 While the predicted rating is 6.
49056 1458175
The actual rating for user 49056 on movie 1458175 is 4.
 While the predicted rating is 5.
28599 103639
The actual rating for user 28599 on movie 103639 is 8.
 While the predicted rating is 4.
50593 1560985
The actual rating for user 50593 on movie 1560985 is 4.
 While the predicted rating is 4.
29000 287978


IndexError: index 0 is out of bounds for axis 0 with size 0

In [26]:
predict_rating(user_mat, movie_mat, 9824982739, 1560985)

IndexError: index 0 is out of bounds for axis 0 with size 0

** A statement about why you think what happened happened.**

In [40]:
reviews.index[reviews.user_id==49056][0]

650566

## Can only predict current users and current movies